# **Classification Using Artificial Neural Networks with Hyperparameter Tuning on Alphabets Data**

# 1.Data Exploration and Preprocessing

In [1]:
# Load the dataset
from google.colab import files
uploaded = files.upload()

Saving Alphabets_data.csv to Alphabets_data.csv


In [2]:
#First, we'll load the necessary libraries and explore the dataset.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Read the dataset
data = pd.read_csv('Alphabets_data.csv')

# Summarize the key features of the dataset
print("Number of samples:", data.shape[0])
print("Number of features:", data.shape[1] - 1)  # excluding the target variable
print("Number of classes:", len(np.unique(data['letter'])))

# Check for missing values
print("Missing values: \n", data.isnull().sum())

Number of samples: 20000
Number of features: 16
Number of classes: 26
Missing values: 
 letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64


In [3]:
# Next, we'll perform data normalization using StandardScaler from scikit-learn.
# Split the dataset into features (X) and target (y)
X = data.drop('letter', axis=1)
y = data['letter']

# Normalize the features(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Converting Target(y) variable into numerical
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
le = LabelEncoder()

# Fit the LabelEncoder to the target variable and transform it
y_num = le.fit_transform(y)

# 2. Model Implementation

In [4]:
from keras.models import Sequential
from keras.layers import Dense

# Define the model architecture
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_num)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_num, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1700 - loss: 2.9514 - val_accuracy: 0.5170 - val_loss: 1.8372
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5817 - loss: 1.5966 - val_accuracy: 0.6860 - val_loss: 1.1571
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7019 - loss: 1.0838 - val_accuracy: 0.7365 - val_loss: 0.9287
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7466 - loss: 0.8959 - val_accuracy: 0.7742 - val_loss: 0.8044
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7722 - loss: 0.7850 - val_accuracy: 0.8002 - val_loss: 0.7179
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8000 - loss: 0.6891 - val_accuracy: 0.8213 - val_loss: 0.6558
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8153 - loss: 0.6407 - val_accuracy: 0.8310 - val_loss: 0.6097
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8290 - loss: 0.5872 - val_accuracy: 0.8455 - val_

# Hyperparameter Tuning

In [5]:
!pip install scikeras
from scikeras.wrappers import KerasClassifier # Import KerasClassifier from scikeras.wrappers
from sklearn.model_selection import GridSearchCV
import numpy as np # Import numpy

# Define the hyperparameter space
param_grid = {
    'epochs': [10, 15], # Pass epochs directly
    'batch_size': [64, 128], # Pass batch_size directly
    'model__optimizer': ['adam', 'sgd', 'rmsprop'], # Keep model__ prefix for arguments specific to create_model function
    'model__activation': ['relu', 'tanh', 'sigmoid'] # Keep model__ prefix for arguments specific to create_model function
}

# Define the model architecture as a function
def create_model(optimizer='adam', activation='relu'):
    from tensorflow import keras # Import keras inside the function to avoid potential conflicts
    from keras.models import Sequential
    from keras.layers import Dense

    model = Sequential()
    model.add(Dense(64, activation=activation, input_shape=(X_scaled.shape[1],)))
    model.add(Dense(32, activation=activation))
    model.add(Dense(len(np.unique(y_num)), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Perform hyperparameter tuning
keras_model = KerasClassifier(model=create_model) # Use KerasClassifier and pass the model function using 'model' argument
grid_search = GridSearchCV(keras_model, param_grid, cv=3)
grid_search.fit(X_train, y_train, epochs=5, batch_size=128)  # Pass fit parameters here


# Print the best hyperparameters and the corresponding score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1392 - loss: 3.1022
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4551 - loss: 2.1138
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6430 - loss: 1.3588
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7052 - loss: 1.0709
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7419 - loss: 0.9207
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1012 - loss: 3.1601
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4246 - loss: 2.2075
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5997 - loss: 1.4981
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6784 - loss: 1.1703
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7174 - loss: 0.9958
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5


...

84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0455 - loss: 3.2846
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1869 - loss: 3.1439
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3018 - loss: 2.9718
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3591 - loss: 2.7345
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3994 - loss: 2.4691
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0629 - loss: 3.2660
Epoch 2/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2010 - loss: 3.1333
Epoch 3/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3324 - loss: 2.9543
Epoch 4/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3659 - loss: 2.6981
Epoch 5/5
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4230 - loss: 2.4464
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1513 - loss: 3.0715
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5639 - loss: 1.7007
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7012 - loss: 1.0864
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7412 - loss: 0.8892
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7740 - loss: 0.7664
Best hyperparameters: {'batch_size': 64, 'epochs': 15, 'model__activation': 'relu', 'model__optimizer': 'adam'}
Best score: 0.7496878495850873


# 4. Evaluation

In [9]:
# Evaluate the tuned model on the test set
y_pred = grid_search.best_estimator_.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='macro'))
print("Recall:", recall_score(y_test, y_pred, average='macro'))
print("F1-score:", f1_score(y_test, y_pred, average='macro'))

# Compare the performance of the default model and the tuned model
print("\n")
default_model = create_model()
default_model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))
y_pred_default = default_model.predict(X_test)

# Convert probabilities to class labels using argmax
y_pred_default = np.argmax(y_pred_default, axis=1)  # Select the class with the highest probability

print("\nDefault model accuracy:", accuracy_score(y_test, y_pred_default))
print("Default model precision:", precision_score(y_test, y_pred_default, average='macro'))
print("Default model recall:", recall_score(y_test, y_pred_default, average='macro'))
print("Default model F1-score:", f1_score(y_test, y_pred_default, average='macro'))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.799
Precision: 0.8015931504016363
Recall: 0.798984676975645
F1-score: 0.7968395661788089




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.1665 - loss: 2.9922 - val_accuracy: 0.4940 - val_loss: 1.9455
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5527 - loss: 1.6893 - val_accuracy: 0.6815 - val_loss: 1.1751
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6881 - loss: 1.1219 - val_accuracy: 0.7418 - val_loss: 0.9346
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7439 - loss: 0.8966 - val_accuracy: 0.7815 - val_loss: 0.8062
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7701 - loss: 0.8024 - val_accuracy: 0.7925 - val_loss: 0.7202
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7930 - loss: 0.7248 - val_accuracy: 0.8108 - val_loss: 0.6627
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8186 - loss: 0.6356 - val_accuracy: 0.8257 - val_loss: 0.6056
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8278 - loss: 0.6068 - val_accuracy: 0.



```
                                                                       SUBMITTED BY: VAMSI KRISHNA GOPIDESI
```

